In [1]:
%run custom_datasets.ipynb

Imported classes.


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import numpy as np
import skimage
import matplotlib.pyplot as plt

import time
import pickle

from datetime import datetime

## Network initialization

In [3]:
class SineLayer(nn.Module):
    # See paper sec. 3.2, final paragraph, and supplement Sec. 1.5 for discussion of omega_0.
    
    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the 
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a 
    # hyperparameter.
    
    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of 
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)
    
    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30, add_dropout=True):
        super().__init__()
        
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = in_features
        self.add_dropout = add_dropout
        
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        self.dropout = nn.Dropout(.2)
        
        self.init_weights()
    
    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                             1 / self.in_features)      
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0, 
                                             np.sqrt(6 / self.in_features) / self.omega_0)
        
    def forward(self, input):
        out = torch.sin(self.omega_0 * self.linear(input))
        if self.add_dropout and not self.is_first: 
            out = self.dropout(out)
        return out
    
    
class Siren(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, 
                 first_different_init=True, outermost_linear=False, 
                 first_omega_0=30, hidden_omega_0=30.):
        super().__init__()
        
        self.net = []
        
        
        # first layer 
        if first_different_init:
            self.net.append(SineLayer(in_features, hidden_features, 
                                  is_first=True, omega_0=first_omega_0))
        else: 
            self.net.append(SineLayer(in_features, hidden_features, omega_0=hidden_omega_0))

            
        # hidden layers
        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features, omega_0=hidden_omega_0))

            
        # last layer
        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)
            
            with torch.no_grad():
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0, 
                                              np.sqrt(6 / hidden_features) / hidden_omega_0)    
            
            self.net.append(final_linear)
        
        else:
            self.net.append(SineLayer(hidden_features, out_features, omega_0=hidden_omega_0))
        
        
        # add all to network
        self.net = nn.Sequential(*self.net)
    
    def forward(self, coords):
        output = self.net(coords)
        return output


def set_device():
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('----------------------------------')
    print('Using device for training:', DEVICE)
    print('----------------------------------')
    
    return DEVICE 

def mem(step):
    div = 1024*1024*1024
    
    t = torch.cuda.get_device_properties(0).total_memory
    c = torch.cuda.memory_cached(0)
    a = torch.cuda.memory_allocated(0)

    print(step)
    print("Memory cached:", round(c/div, 3))
    print("Memory allocated:", round(c/div, 3))
    print()

In [4]:
DEVICE = set_device()


----------------------------------
Using device for training: cuda
----------------------------------


In [5]:
data = PrepareData3D(["Aorta Volunteers", "Aorta BaV", "Aorta Resvcue", "Aorta CoA"])

train_ds = SirenDataset(data.train, DEVICE) 
train_dataloader = DataLoader(train_ds, batch_size=1, num_workers=0)

In [6]:
val_ds = SirenDataset(data.val, DEVICE) 
val_dataloader = DataLoader(val_ds, batch_size=1, num_workers=0)

In [7]:
train_ds[0]

(0,
 'RESV_109.npy',
 'Aorta Resvcue',
 tensor([[-1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -0.9130],
         [-1.0000, -1.0000, -0.8261],
         ...,
         [ 1.0000,  1.0000,  0.8261],
         [ 1.0000,  1.0000,  0.9130],
         [ 1.0000,  1.0000,  1.0000]], device='cuda:0'),
 tensor([[-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [-1.]], device='cuda:0'),
 tensor([[-1.],
         [-1.],
         [-1.],
         ...,
         [-1.],
         [-1.],
         [-1.]], device='cuda:0'))

### Init models

In [8]:
size = 256

def create_model_per_image(dataset): 
    models = {}
    
    for i in range(dataset.__len__()):
        subj = dataset[i][1].item()
        
        models[subj] = []

        model = Siren(in_features=3, out_features=size, hidden_features=size, 
                      hidden_layers=1, first_different_init=True, outermost_linear=False).cuda()

        models[subj].append(model)
        models[subj].append(torch.optim.Adam(lr=1e-4, params=model.parameters()))
    
    return models


In [9]:
pcmra_siren = Siren(in_features=size, out_features=1, hidden_features=size, 
                  hidden_layers=1, first_different_init=False, outermost_linear=True).cuda()
pcmra_optim = torch.optim.Adam(lr=1e-4, params=pcmra_siren.parameters())

mask_siren = Siren(in_features=size, out_features=1, hidden_features=size, 
                  hidden_layers=1, first_different_init=False, outermost_linear=True).cuda()
mask_optim = torch.optim.Adam(lr=1e-4, params=mask_siren.parameters())

train_models = create_model_per_image(train_ds)

val_models = create_model_per_image(val_ds)

train_models["RESV_109.npy"]

[Siren(
   (net): Sequential(
     (0): SineLayer(
       (linear): Linear(in_features=3, out_features=256, bias=True)
       (dropout): Dropout(p=0.2, inplace=False)
     )
     (1): SineLayer(
       (linear): Linear(in_features=256, out_features=256, bias=True)
       (dropout): Dropout(p=0.2, inplace=False)
     )
     (2): SineLayer(
       (linear): Linear(in_features=256, out_features=256, bias=True)
       (dropout): Dropout(p=0.2, inplace=False)
     )
   )
 ),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.0001
     weight_decay: 0
 )]

### Train and validation functions

In [10]:
def l2_loss(out, ground_truth): 
    return ((out - ground_truth)**2).mean()

def train_model(coords, ground_truth, h_model, h_optim, 
                t_model, t_optim, criterion, train_head=True, train_tail=True): 

    out = h_model(coords)
    out = t_model(out)

    loss = criterion(out, ground_truth)

    loss.backward()

    if train_head: 
        h_optim.step()
    
    if train_tail: 
        t_optim.step()
    
    h_optim.zero_grad()
    t_optim.zero_grad()
    
    return out, loss

In [11]:
train_models = pickle.load(open("Models/train_models_05_03_2021_20_15_03..pkl", "rb"))
pcmra_siren = pickle.load(open("Models/pcmra_siren_05_03_2021_20_15_03..pkl", "rb"))
mask_siren = pickle.load(open("Models/mask_siren_05_03_2021_20_15_03..pkl", "rb"))

FileNotFoundError: [Errno 2] No such file or directory: 'Models/train_models_05_03_2021_20_15_03..pkl'

In [ ]:
batches = 1000
steps_til_summary = 10

now = datetime.now()
dt = now.strftime("%d_%m_%Y_%H_%M_%S")
print(dt)

lowest_losses = 1000
for batch in range(batches):
    
    pcmra_losses = []
    mask_losses = []
        
    for idx, subj, proj, coords, pcmra, mask in train_dataloader:
        subj = subj[0].item()
        img_siren, img_optim = train_models[subj]
        
        pcmra_out, pcmra_loss = train_model(coords, pcmra, img_siren, img_optim, 
                                 pcmra_siren, pcmra_optim, l2_loss)
        
        mask_out, mask_loss = train_model(coords, mask, img_siren, img_optim,
                                mask_siren, mask_optim, l2_loss)
        
        
        pcmra_losses.append(pcmra_loss.item())
        mask_losses.append(mask_loss.item())
    
    if not batch % steps_til_summary:
        p_mean, p_std = round(np.mean(pcmra_losses), 6), round(np.std(pcmra_losses), 6)
        m_mean, m_std = round(np.mean(mask_losses), 6), round(np.std(mask_losses), 6)
        
        print(f"Batch {batch} \n Pcmra loss: \t mean {p_mean} \t std {p_std} \n Mask loss: \t mean {m_mean} \t std {m_std}")
        
        if (p_mean + m_mean) < lowest_losses: 
            print("Saving models...")
            lowest_losses = (p_mean + m_mean)
            pickle.dump(train_models, open(f"train_models_{dt}.pkl", "wb"))
            pickle.dump(pcmra_siren, open(f"pcmra_siren_{dt}.pkl", "wb"))
            pickle.dump(mask_siren, open(f"mask_siren_{dt}.pkl", "wb"))
#             print("Models saved")

### Train single Image Siren

In [ ]:
# torch.cuda.empty_cache()

batches = 500
steps_til_summary = 10

lowest_losses = 1000

for batch in range(batches):
    
#     idx, subj, proj, coords, pcmra, mask = train_ds[3]
    idx, subj, proj, coords, pcmra, mask = val_ds[1]
    subj = subj.item()
    
#     img_siren, img_optim = train_models[subj]
    img_siren, img_optim = val_models[subj]
    
    pcmra_out, pcmra_loss = train_model(coords, pcmra, img_siren, img_optim, 
                                        pcmra_siren, pcmra_optim, l2_loss, 
                                        train_tail=False)

    mask_out, mask_loss = train_model(coords, mask, img_siren, img_optim,
                                    mask_siren, mask_optim, l2_loss, 
                                    train_head=True, train_tail=False)
    
    if not batch % steps_til_summary:
        print(f"Batch {batch} \t pcmra loss: {round(pcmra_loss.item(), 6)} \t mask loss: {round(mask_loss.item(), 6)}")

In [ ]:
slic = 10

fig, axes = plt.subplots(2,2, figsize=(12,12))
axes[0, 0].imshow(pcmra_out.cpu().view(128, 128, 24).detach().numpy()[:, :, slic])
axes[0, 1].imshow(pcmra.cpu().view(128, 128, 24).detach().numpy()[:, :, slic])
axes[1, 0].imshow(mask_out.cpu().view(128, 128, 24).detach().numpy().round()[:, :, slic])
axes[1, 1].imshow(mask.cpu().view(128, 128, 24).detach().numpy()[:, :, slic])

plt.show()


In [ ]:
img_siren = 0

In [ ]:
torch.cuda.empty_cache()


In [ ]:
# prints currently alive Tensors and Variables
import torch
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

In [ ]:
train_models = 0
val_models=0